In [1]:
from bs4 import BeautifulSoup
import requests
from pyth.plugins.rtf15.reader import Rtf15Reader
from pyth.plugins.xhtml.writer import XHTMLWriter
import pandas as pd
import urllib2
from cStringIO import StringIO

import re

#All the rtf links
url = "https://www.gov.uk/government/statistics/prison-population-figures-2015"
r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")
s1 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [2]:
url = "https://www.gov.uk/government/statistics/prison-population-figures-2016"

r = requests.get(url)
soup = BeautifulSoup(r.content,"lxml")
s2 = set([(a.text, a["href"]) for a in soup.findAll("a") if ".rtf" in a["href"] and a.text != ""])

In [3]:
all_links = s1.union(s2)

In [4]:
def get_table(doc, source):
    final_data = []

    header = False
    for c in doc.content:
        full_p = "".join([d.content[0] for d in c.content])
        full_p = full_p.replace("\t**", "")
        if "Capacity" in full_p and "Use CNA" in full_p:
            
            header = True
            continue

        if header:
            row= re.split(r"\t", full_p)
            if len(row) == 7 and "Prison Name" not in row:
                final_data.append(row)
    df = pd.DataFrame(final_data, columns = ["prison_name","baseline_cna", "in_use_cna", "operational_capacity", "population", "perc_pop_to_used_cna", "perc_acc_available"])
    df.iloc[:,1:] = df.iloc[:,1:].replace("%", "", regex=True).replace(",", "", regex=True)

    for c in df.columns:
        df[c]= pd.to_numeric(df[c], errors='ignore')
    cols = [c for c in df.columns if "perc" in c]
    df.loc[:,cols] = df.loc[:,cols]/100
    
    all_text =  "\n".join([p.content[0].content[0] for p in doc.content])
    
    #Parse out the date here
    re_pattern = r"(\d{2}/\d{2}/\d{4})"
    
    date = re.search(re_pattern, all_text).group(1)   
    df["date"] = date
    df["source"] = source
    
    
    return df

In [5]:
import urllib2
import os

dfs = []

for link in all_links:
    text = link[0]
    link = link[1]
    rtf_file = urllib2.urlopen("http://www.gov.uk" +link)
    rtf_file = StringIO(rtf_file.read())
    doc = Rtf15Reader.read(rtf_file)
    df = get_table(doc, "http://www.gov.uk" +link)
    dfs.append(df)
    
    head, tail = os.path.split(link)
    rtf_file = urllib2.urlopen("http://www.gov.uk" +link)
    with open("all_rtf/" + tail,'wb') as output:
        output.write(rtf_file.read())

In [6]:
final = pd.concat(dfs)
final["date"] = pd.to_datetime(final["date"],infer_datetime_format=True)

In [7]:
final.to_csv("pop_data/all_pop_data_rtf.csv", index=False, encoding="utf-8")

In [8]:
dfs = []
dfs.append(pd.read_csv("pop_data/all_pop_data_rtf.csv"))
dfs.append(pd.read_csv("pop_data/all_pop_data_txt.csv"))

all_data = pd.concat(dfs)
pgs = pd.read_csv("manual/prison_groups_lookup_and_manual_classification.csv")

In [9]:
pgs = pgs.drop([c for c in pgs.columns if "Unnamed" in c],axis=1)
pgs = pgs.fillna(0)

In [10]:
all_data = all_data.merge(pgs[["prison_name_original","prison_name_group"]], left_on="prison_name", right_on="prison_name_original", how="left")
all_data = all_data.fillna(0)
all_data.head(1)

,prison_name,baseline_cna,in_use_cna,operational_capacity,population,perc_pop_to_used_cna,perc_acc_available,date,source,prison_name_original,prison_name_group
0,Altcourse,794,794,1133,1113,1.4,1,2015-02-27,http://www.gov.uk/government/uploads/system/up...,Altcourse,Altcourse


In [11]:
all_data = all_data.drop(["prison_name", "prison_name_original"], axis=1)
all_data.head(2)
all_data  = all_data.pivot_table(index=["prison_name_group", "source", "date"], aggfunc="sum", values = ["operational_capacity", "baseline_cna", "in_use_cna","population"]).reset_index()
prisons = pd.read_csv("manual/all_prison_data_final_manual.csv")
prisons["lat"]= prisons["lat"].fillna(prisons["lat_manual"])
prisons["lng"]= prisons["lng"].fillna(prisons["lng_manual"])

IOError: File manual/all_prison_data_final_manual.csv does not exist

In [ ]:
prisons = prisons[["moj_prison_name", "lat", "lng", "nomis_id"]]
final_pop_data = all_data.merge(prisons, left_on="prison_name_group", right_on="moj_prison_name")
final_pop_data.head()
final_pop_data["perc_pop_to_used_cna"] = final_pop_data["population"]*1.0/final_pop_data["in_use_cna"]*1.0
final_pop_data["perc_acc_available"] = final_pop_data["in_use_cna"]*1.0/final_pop_data["baseline_cna"]*1.0

In [ ]:
# join on meta data
metas = pgs.drop_duplicates("prison_name_group")
metas = metas.drop(["prison_name_original", "prison_meta_info_source", "closed", "done"], axis=1)
final_pop_data = final_pop_data.merge(metas, left_on = "prison_name_group", right_on="prison_name_group", how="left")

In [ ]:
final_pop_data = final_pop_data.sort(["moj_prison_name", "date"])
final_pop_data = final_pop_data.drop("prison_name_group", axis=1)
final_pop_data.to_csv("pop_data/final_full_time_series.csv", index=False)
final_pop_data.to_csv("../web/data/pop_data/final_full_time_series.csv", index=False)